In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [55]:
data = pd.read_csv('UNdata_Export_20181124_222557311.csv')

In [56]:
data = data.drop(columns=['Value Footnotes'])

In [57]:
data.head(5)

,Country or Area,Year,Value
0,Afghanistan,2014,6.390000
1,Afghanistan,2013,5.900000
2,Afghanistan,2012,5.454545
3,Afghanistan,2011,5.000000
4,Afghanistan,2010,4.000000


In [63]:
# Create x, where x the 'scores' column's values as floats
x = data[['Value']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_normalized['normalized'] = pd.DataFrame(x_scaled)
df_normalized.drop(columns=0,inplace=True)

In [65]:
data_new = pd.concat([data,df_normalized ], axis=1)
data_new.head()

,Country or Area,Year,Value,normalized
0,Afghanistan,2014,6.390000,0.065098
1,Afghanistan,2013,5.900000,0.060106
2,Afghanistan,2012,5.454545,0.055568
3,Afghanistan,2011,5.000000,0.050937
4,Afghanistan,2010,4.000000,0.040750


In [66]:
data_new.columns

Index(['Country or Area', 'Year', 'Value', 'normalized'], dtype='object')

In [47]:
ser = data['Country or Area'].value_counts()

In [51]:
data_countries = pd.read_excel('Developing countries.xlsx')

developed_countries = data_countries [data_countries['Value'] == 0]
developing_countries = data_countries [data_countries['Value'] == 1] 

list_developed_countries = developed_countries['Countries']
list_developing_countries = developing_countries['Countries']

list_developed_countries = list(list_developed_countries)
list_developing_countries = list(list_developing_countries)

In [52]:
data['label'] = -1

In [61]:
for i,da in data.iterrows():
    if da['Country or Area'] in list_developed_countries:
        val = 1
        data.at[i,'label'] = val
    elif da['Country or Area'] in list_developing_countries:
        val = 0
        data.at[i,'label'] = val

In [67]:
data.columns

Index(['Country or Area', 'Year', 'Value', 'label'], dtype='object')

In [68]:
list_x = ['Country or Area', 'Year', 'Value']
X = data[list_x]
y = data['label']

In [71]:
from tsfresh import extract_features
extracted_features = extract_features(X, column_id="Country or Area", column_sort="Year")

Feature Extraction: 100%|██████████| 10/10 [00:19<00:00,  1.79s/it]


In [72]:
extracted_features.head()

variable,Value__abs_energy,Value__absolute_sum_of_changes,"Value__agg_autocorrelation__f_agg_""mean""","Value__agg_autocorrelation__f_agg_""median""","Value__agg_autocorrelation__f_agg_""var""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Value__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Value__time_reversal_asymmetry_statistic__lag_1,Value__time_reversal_asymmetry_statistic__lag_2,Value__time_reversal_asymmetry_statistic__lag_3,Value__value_count__value_-inf,Value__value_count__value_0,Value__value_count__value_1,Value__value_count__value_inf,Value__value_count__value_nan,Value__variance,Value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
Afghanistan,171.949738,6.919847,-0.587144,-0.513169,0.919234,4.0,1.0,2.39,0.0,NaN,...,25.548749,48.237516,63.322128,0.0,0.0,0.0,0.0,0.0,5.081028,1.0
Albania,16925.889951,59.774202,-0.553129,-0.762668,0.820387,45.0,1.0,15.10,0.0,NaN,...,22236.028783,44042.108801,64061.130082,0.0,0.0,0.0,0.0,0.0,526.237172,1.0
Algeria,1620.516765,17.443886,-0.644761,-0.473405,1.142363,12.5,1.0,5.59,0.0,NaN,...,573.471373,1077.129102,1528.656443,0.0,0.0,0.0,0.0,0.0,30.232650,1.0
Andorra,59560.427059,86.299531,-0.588723,-0.495535,0.910084,81.0,1.0,14.90,0.0,NaN,...,103018.876022,205434.535786,328862.707219,0.0,0.0,0.0,0.0,0.0,832.723356,1.0
Angola,1494.788330,21.123986,-0.538600,-0.569763,0.750344,10.0,1.0,11.26,0.0,NaN,...,907.365714,1533.373306,1628.234586,0.0,0.0,0.0,0.0,0.0,55.580544,1.0


In [73]:
extracted_features.shape

(210, 794)

In [74]:
lsi_cols_with_nan = extracted_features.columns[extracted_features.isna().any()].tolist()

In [75]:
len(lsi_cols_with_nan)

552

In [77]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
data_after_impute = impute(extracted_features)

 'Value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'Value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'Value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'Value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'Value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'Value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'Value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'Value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'Value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'Value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'Value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'Value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'Value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"rvalue"'
 'Value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"slope"'
 'Value__agg_linear_trend

In [78]:
lsi_cols_with_nan_after_impute = data_after_impute.columns[data_after_impute.isna().any()].tolist()

In [79]:
len(lsi_cols_with_nan_after_impute)

0

In [80]:
data_after_impute.head()

variable,Value__abs_energy,Value__absolute_sum_of_changes,"Value__agg_autocorrelation__f_agg_""mean""","Value__agg_autocorrelation__f_agg_""median""","Value__agg_autocorrelation__f_agg_""var""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Value__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Value__time_reversal_asymmetry_statistic__lag_1,Value__time_reversal_asymmetry_statistic__lag_2,Value__time_reversal_asymmetry_statistic__lag_3,Value__value_count__value_-inf,Value__value_count__value_0,Value__value_count__value_1,Value__value_count__value_inf,Value__value_count__value_nan,Value__variance,Value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
Afghanistan,171.949738,6.919847,-0.587144,-0.513169,0.919234,4.0,1.0,2.39,0.0,0.0,...,25.548749,48.237516,63.322128,0.0,0.0,0.0,0.0,0.0,5.081028,1.0
Albania,16925.889951,59.774202,-0.553129,-0.762668,0.820387,45.0,1.0,15.10,0.0,0.0,...,22236.028783,44042.108801,64061.130082,0.0,0.0,0.0,0.0,0.0,526.237172,1.0
Algeria,1620.516765,17.443886,-0.644761,-0.473405,1.142363,12.5,1.0,5.59,0.0,0.0,...,573.471373,1077.129102,1528.656443,0.0,0.0,0.0,0.0,0.0,30.232650,1.0
Andorra,59560.427059,86.299531,-0.588723,-0.495535,0.910084,81.0,1.0,14.90,0.0,0.0,...,103018.876022,205434.535786,328862.707219,0.0,0.0,0.0,0.0,0.0,832.723356,1.0
Angola,1494.788330,21.123986,-0.538600,-0.569763,0.750344,10.0,1.0,11.26,0.0,0.0,...,907.365714,1533.373306,1628.234586,0.0,0.0,0.0,0.0,0.0,55.580544,1.0


In [85]:
data_after_impute.to_csv('new.csv')

In [86]:
new_df = pd.read_csv('new.csv')

In [87]:
new_df.head()

,id,Value__abs_energy,Value__absolute_sum_of_changes,"Value__agg_autocorrelation__f_agg_""mean""","Value__agg_autocorrelation__f_agg_""median""","Value__agg_autocorrelation__f_agg_""var""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""",...,Value__time_reversal_asymmetry_statistic__lag_1,Value__time_reversal_asymmetry_statistic__lag_2,Value__time_reversal_asymmetry_statistic__lag_3,Value__value_count__value_-inf,Value__value_count__value_0,Value__value_count__value_1,Value__value_count__value_inf,Value__value_count__value_nan,Value__variance,Value__variance_larger_than_standard_deviation
0,Afghanistan,171.949738,6.919847,-0.587144,-0.513169,0.919234,4.0,1.0,2.39,0.0,...,25.548749,48.237516,63.322128,0.0,0.0,0.0,0.0,0.0,5.081028,1.0
1,Albania,16925.889951,59.774202,-0.553129,-0.762668,0.820387,45.0,1.0,15.10,0.0,...,22236.028783,44042.108801,64061.130082,0.0,0.0,0.0,0.0,0.0,526.237172,1.0
2,Algeria,1620.516765,17.443886,-0.644761,-0.473405,1.142363,12.5,1.0,5.59,0.0,...,573.471373,1077.129102,1528.656443,0.0,0.0,0.0,0.0,0.0,30.232650,1.0
3,Andorra,59560.427059,86.299531,-0.588723,-0.495535,0.910084,81.0,1.0,14.90,0.0,...,103018.876022,205434.535786,328862.707219,0.0,0.0,0.0,0.0,0.0,832.723356,1.0
4,Angola,1494.788330,21.123986,-0.538600,-0.569763,0.750344,10.0,1.0,11.26,0.0,...,907.365714,1533.373306,1628.234586,0.0,0.0,0.0,0.0,0.0,55.580544,1.0


In [88]:
new_df['label'] = -1

In [90]:
for i,da in new_df.iterrows():
    if da['id'] in list_developed_countries:
        val = 1
        new_df.at[i,'label'] = val
    elif da['id'] in list_developing_countries:
        val = 0
        new_df.at[i,'label'] = val

In [94]:
y = new_df['label']
X = new_df.drop(columns=['id','label'])

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [151]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)

In [138]:
dsf = classification_report(y_test,pred)

In [139]:
print(dsf)

             precision    recall  f1-score   support

         -1       0.53      0.32      0.40        25
          0       0.57      0.78      0.66        27
          1       0.64      0.64      0.64        11

avg / total       0.57      0.57      0.55        63



In [140]:
acc = accuracy_score(y_test,pred) 

In [141]:
acc

0.5714285714285714

In [149]:
rf_classifier = RandomForestClassifier(n_estimators=10, random_state = 42)
rf_classifier.fit(X_train,y_train)
pred = rf_classifier.predict(X_test)
print("F1 Score: ", metrics.f1_score(y_test,pred, average='weighted'))
print("Accuracy Score: ", accuracy_score(y_test,pred))

F1 Score:  0.5404828598105909
Accuracy Score:  0.5396825396825397
